# cr3 CTF 2024
##    dummy-aes   | 474 pts

Task description:

```
Cocay just sent me his new AES, please help me break it so i can 'haha, skill issue' him xD.

$ nc 1337.sb 20003

Author: Onirique
```

Attachments: [main.py](./main.py), [aes.py](./aes.py)

## Solution

Well, it's almost regular aes implementation. The only change is the matrix multiplication in the last round:

In [ ]:
        for i in range(1, 10):
            self.__round_encrypt(self.plain_state, self.round_keys[4 * i: 4 * (i + 1)])

        self.plain_state = self.__mat_multiplication(self.plain_state, self.mul_matrix) # !!!!!!!!!!!!!!!!
        self.__sub_bytes(self.plain_state)
        self.__shift_rows(self.plain_state)
        self.__add_round_key(self.plain_state, self.round_keys[40:])

So, we are allowed to add errors in the last round and everything else is linear. So I decided to send identity matrix and - identity matrix and then recover the last round key.

Suppose $k_{00}$ is the first byte of the last round key.

Then: $ct_{00} = Shift(Sbox[\alpha * st_{l - 1}]) \bigoplus k_{00}$, where $\alpha$ is my scalar of identity matrix.

So, if we send $I$ and $\alpha I$ then for true $k_{00}$:  $InvSbox[ct_{00} \bigoplus k_{00}] * \alpha = InvSbox[ct1_{00} \bigoplus k_{00}]$

Thus we can determine if the key is suitable.

Also, only one scalar is not enough, so I sent $3 I$ too.

After recovering the last round key, we can simply reverse the key schedule procedure to recover master key and decrypt everything.

For simplicity I sent identity matrix for flag encryption to use the Crypto.Cipher AES.

In [2]:
Sbox = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

InvSbox = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)

Rcon = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)

def text2matrix(text):
    matrix = []
    for i in range(16):
        byte = (text >> (8 * (15 - i))) & 0xFF
        if i % 4 == 0:
            matrix.append([byte])
        else:
            matrix[i // 4].append(byte)
    return matrix


def matrix2text(matrix):
    text = 0
    for i in range(4):
        for j in range(4):
            text |= (matrix[i][j] << (120 - 8 * (4 * i + j)))
    return text

In [5]:
from pwn import remote, context
import re
context.log_level = "error"

In [6]:
host, port = "1337.sb", 20003

identity = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
negidentity = [[255, 0, 0, 0], [0, 255, 0, 0], [0, 0, 255, 0], [0, 0, 0, 255]]
tridentity = [[3, 0, 0, 0], [0, 3, 0, 0], [0, 0, 3, 0], [0, 0, 0, 3]]

r = remote(host, port)
r.recvline()

ms = []
for resm in [identity, negidentity, tridentity]:
    for _ in range(4):
        print(r.recvline())

    r.sendline(b'1')
    r.sendline((b'\x00' * 15).hex().encode()) # encrypt there whatever you want
    r.sendline(matrix2text(resm).to_bytes(16, 'big').hex().encode())
    m = r.recvline().decode()
    print(m.encode())
    m = int(re.findall(r'Enter your matrix\(hex\): (.*)\n', m)[0])
    ms.append(text2matrix(m))

for _ in range(4):
    print(r.recvline())

r.sendline(b'2') # obtain the flag encryption using identity matrix
r.sendline(matrix2text(identity).to_bytes(16, 'big').hex().encode())
m = r.recvline().decode()
enc_flag = int(re.findall(r'Enter your matrix\(hex\): (.*)\n', m)[0])
r.close()

b'==== MENU ====\n'
b'1. Encrypt your plaintext\n'
b'2. Get encrypted flag\n'
b'3. Quit\n'
b'> Enter your plaintext(hex): Enter your matrix(hex): 94830128947278342324761026234531953406\n'
b'==== MENU ====\n'
b'1. Encrypt your plaintext\n'
b'2. Get encrypted flag\n'
b'3. Quit\n'
b'> Enter your plaintext(hex): Enter your matrix(hex): 313943871054136115201138139031867887318\n'
b'==== MENU ====\n'
b'1. Encrypt your plaintext\n'
b'2. Get encrypted flag\n'
b'3. Quit\n'
b'> Enter your plaintext(hex): Enter your matrix(hex): 259240790096986079673453044779070560145\n'
b'==== MENU ====\n'
b'1. Encrypt your plaintext\n'
b'2. Get encrypted flag\n'
b'3. Quit\n'


Now we start the recovery process

In [8]:
from itertools import product

possible_last_round_key = []
for i, j in product([0, 1, 2, 3], repeat=2):
    tmp = []
    for k in range(256):
        t1 = InvSbox[ms[0][i][j] ^ k]
        t2 = InvSbox[ms[1][i][j] ^ k]
        t3 = InvSbox[ms[2][i][j] ^ k]
        if t1 == 256 - t2 and (3 * t1) % 256 == t3:
            tmp.append(k)
    print(tmp)
    possible_last_round_key.append(tmp)

[226]
[194]
[81]
[117]
[189]
[84]
[214]
[44]
[246]
[249]
[27]
[57]
[17]
[255]
[93]
[131]


We can see that with only 2 errors we have found a unique key. When I used only $-I$ there were like $4^16$ possible keys.

In [9]:
from Crypto.Cipher import AES
for k in product(*possible_last_round_key): # iterating over all the possible keys

    # reverse the key schedule
    l_k = [[k[j + 4 * i] for j in range(4)] for i in range(4)]
    round_keys = [[0] * 4 for _ in range(4 * 10)] + l_k
    
    for i in reversed(range(4, 4 * 11)):
        if i % 4 == 0:
            for j in reversed(range(1, 4)):
                byte = round_keys[i][j]
                round_keys[i - 4][j] = byte ^ Sbox[round_keys[i-1][(j +1) % 4]]

            byte = round_keys[i][0]
            round_keys[i - 4][0] = byte ^ Sbox[round_keys[i - 1][1]] ^ Rcon[i // 4]
        else:
            for j in reversed(range(4)):
                byte = round_keys[i][j]
                round_keys[i - 4][j] = byte ^ round_keys[i - 1][j]

    master_key = round_keys[:4]
    master_key = b"".join(bytes(x) for x in master_key)
    cipher = AES.new(key=master_key, mode=AES.MODE_ECB)
    print(cipher.decrypt(enc_flag.to_bytes(48, 'big')))

b'cr3{T1L_r0und5_k3ys_c4n_b3_r3v3rs3d_OwO}\x08\x08\x08\x08\x08\x08\x08\x08'
